## Setup

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import json

from tests.eas.load_tracking import FreqInvarianceTest
from trace import Trace
from trappy.plotter import plot_trace
from trappy.stats.grammar import Parser
from trappy import ILinePlot

In [ ]:
import logging
from conf import LisaLogging
LisaLogging.setup()
logging.getLogger('Analysis').setLevel(logging.ERROR)
logging.getLogger('Trace').setLevel(logging.ERROR)

## Run test workload

In [ ]:
t = FreqInvarianceTest(methodName='test_task_util')
# help(t)

In [ ]:
t.setUpClass()

## Show variance in util_avg and load_avg

In [ ]:
# Get the frequency an experiment was run at
def experiment_freq(exp):
    [cpu] = exp.wload.cpus
    freq = exp.conf['cpufreq']['freqs'][cpu]
    return freq
freqs = [experiment_freq(e) for e in t.executor.experiments]
freqs

In [ ]:
def plot_signal_against_freq(signal):
    means = [t.get_signal_mean(e, signal) for e in t.executor.experiments]
    limits = (min(means) * 0.99, max(means) * 1.01)
    pd.DataFrame(means, index=freqs, columns=['Mean ' + signal]).plot(kind='bar', ylim=limits)

### Plot of variation of util_avg value with frequency:

In [ ]:
plot_signal_against_freq('util_avg')

### And the same thing for load_avg:

In [ ]:
plot_signal_against_freq('load_avg')

## Examine trace from workload execution

The log `setUpClass` call above shows the experiments that were run. Use the function below to get a plot from a given experiment of the `util_avg` scheduler signal for that workload's tasks, along with the expected mean value.

In [ ]:
plot_trace(t.get_trace(t.executor.experiments[0]).ftrace)

In [ ]:
signals = ['util_avg', 'load_avg']
def examine_experiment(idx):
    experiment = t.executor.experiments[idx]
    
    print "Experiment configuration:" 
    print json.dumps(experiment.conf, indent=4)
    events = t.te.test_conf["ftrace"]["events"]
    
    # todo add get_trace method
    tasks = experiment.wload.tasks.keys()
    #trace = Trace(t.te.platform, experiment.out_dir, events, tasks)
    #print "Trace plot:"
    #plot_trace(trace.ftrace)
    
    # Get observed signal
    signal_df = t.get_sched_signals(experiment, signals)
#    return signal_df
    # Get expected average value for util_avg signal
    expected_util_avg_mean = t.get_expected_util_avg(experiment)
    
    # Plot util avg signal with expected mean value
    util_avg_mean = pd.Series([expected_util_avg_mean], name="expected_util_avg", index=[signal_df.index[0]])
    df = pd.concat([signal_df, util_avg_mean], axis=1).fillna(method="ffill")
    ILinePlot(df, column=signals + ["expected_util_avg"], drawstyle="steps-post").view()

In [ ]:
examine_experiment(0)

In [ ]:
df = examine_experiment(-1, "load_avg")